In [ ]:
# Imports
import time
import copy
import numpy as np
import pandas as pd

import panel as pn
import param

from pymepix.channel.channel_types import ChannelDataType, Commands

from online_data.timepix_monitor.tpx_acquisition import initialize_timepix_monitor, unregister_all
from online_data.timepix_monitor.tpx_acquisition import callback

import holoviews as hv
from holoviews import opts
from holoviews import streams
from holoviews.streams import Pipe
from bokeh.plotting import show

hv.extension('bokeh')
#pn.extension()

In [ ]:
initialize_timepix_monitor()

## Globals ##

In [ ]:
TOF_BINS_NUMBER = 1000
TOF_RANGE = 10.0
TOF_YLOG = True

bins_VMI = (range(0, 256), range(0, 256))
H_VMI = np.zeros((255, 255), dtype=float)

bins_series = np.linspace(0, TOF_RANGE, TOF_BINS_NUMBER)
x_axe = bins_series[:-1]


scan_data = []
scan_table = pd.DataFrame()


full_range_roi_init_data = {
    'full_range':{
        'tof_roi': (0,TOF_RANGE),
        'spatial_roi': ((0,0), (255,255)),
        'roi_tof_histo': np.zeros((len(bins_series)-1), dtype=float),
        'roi_H_VMI': np.zeros((255, 255), dtype=float),
        'roi_color': '#99ef78',
        'n_tiggers': 0
    }
}
    
scan_data.append(full_range_roi_init_data)  #always at least one record data


pipe_roi_scan = Pipe(data=[])
pipe_image = Pipe(data=[])
pipe_tof = Pipe(data=[])

pipe_roi_scan.send(scan_table)
pipe_image.send(H_VMI)
pipe_tof.send(np.zeros((len(bins_series)-1),) )

### Image&Spectrum panel  ###

In [ ]:
def on_roi_select_change(Event):
    pass    

def reset_button_callback(Event):
    global scan_data, scan_table
    
    newRecord = copy.deepcopy(scan_data[-1])
    for roi_name, roi_record in newRecord.items():
        roi_record['roi_tof_histo'] = np.zeros((len(bins_series)-1), dtype=float)
        roi_record['roi_H_VMI'] =  np.zeros((255, 255), dtype=float)
        roi_record['n_tiggers'] = 0
    scan_data=[newRecord]
    scan_table = pd.DataFrame()
    

roi_select = pn.widgets.MultiChoice(
    name='ToF/mass ROI', value=list(scan_data[-1].keys()),
    options=list(scan_data[-1].keys()),)

reset_button = pn.widgets.Button(name='Reset', button_type='primary')
reset_button.on_click(reset_button_callback)

roi_select.param.watch(on_roi_select_change, 'value')

### mass calibration panel ###

In [ ]:
mass_calibration_data = None # otherwise it is tuple(A, t0), m=A(t-t0)

time_point1 = pn.widgets.FloatInput(name='Time 1', value=0., start=0, end=TOF_RANGE)
time_point2 = pn.widgets.FloatInput(name='Time 2', value=0., start=0, end=TOF_RANGE)
mass_point1 = pn.widgets.FloatInput(name='Mass 1', value=0.,)
mass_point2 = pn.widgets.FloatInput(name='Mass 2', value=0.,)

def calculate_mass_from_tof(time_series):
    if mass_calibration_data != None:
        return mass_calibration_data[0]*pow((time_series - mass_calibration_data[1]),2)
    return time_series

def mass_calibrate_callback(Event):
    global mass_calibration_data, bins_series, x_axe
    
    mass1_sqrt = pow(mass_point1.value,0.5)
    mass2_sqrt = pow(mass_point2.value,0.5)
    
    t0 = (mass2_sqrt*time_point1.value - mass1_sqrt*time_point2.value)/(mass2_sqrt-mass1_sqrt)
    A = mass_point2.value/pow((time_point2.value - t0),2)
    
    mass_calibration_data = (A, t0)
    
    mass_calib_string.value = f'{A}  {t0}'
    
    bins_series = np.linspace(0, TOF_RANGE, TOF_BINS_NUMBER)
    bins_series = calculate_mass_from_tof(bins_series)
    
    min_mass_index = np.argmin(bins_series)
    if min_mass_index > 0:
        bins_series[:min_mass_index] = -bins_series[:min_mass_index]
   
    x_axe = bins_series[:-1]

def reset_calibration_callback(Event):
    global mass_calibration_data, bins_series, x_axe
    mass_calibration_data = None
    bins_series = np.linspace(0, TOF_RANGE, TOF_BINS_NUMBER)
    x_axe = bins_series[:-1]
    mass_calib_string.value = 'None'

mass_calibrate_button = pn.widgets.Button(name='Calibrate ToF spectra', button_type='primary')
mass_calibrate_button.on_click(mass_calibrate_callback)

reset_calibration_button = pn.widgets.Button(name='Reset mass calibration', button_type='primary')
reset_calibration_button.on_click(reset_calibration_callback)

mass_calib_string = pn.widgets.StaticText(name='Calibration const.', value='None')



### ROI management panel ###

In [ ]:
m1_point = pn.widgets.FloatInput(name='m1', value=0., start=0)
m2_point = pn.widgets.FloatInput(name='m2', value=0., start=0)

x1_point = pn.widgets.IntInput(name='X1', value=0,   start=0, end=255)
x2_point = pn.widgets.IntInput(name='X2', value=255, start=0, end=255)
y1_point = pn.widgets.IntInput(name='Y1', value=0,   start=0, end=255)
y2_point = pn.widgets.IntInput(name='Y2', value=255, start=0, end=255)

text_input_roi_name = pn.widgets.TextInput(name='ROI name', placeholder='Enter the name of ROI here...')

colorpicker = pn.widgets.ColorPicker(name='ROI color', value='#99ef78')

def add_roi_callback(Event):
    new_roi= {
            'tof_roi': (m1_point.value, m2_point.value),
            'spatial_roi': ((x1_point.value,y1_point.value), (x2_point.value,y2_point.value)),
            'roi_tof_histo': np.zeros((len(bins_series)-1), dtype=float),
            'roi_H_VMI': np.zeros((255, 255), dtype=float),
            'roi_color': colorpicker.value,
            'n_tiggers': 0
    }
    
    scan_data[-1][text_input_roi_name.value] = new_roi
    
    if not (text_input_roi_name.value in roi_select.options):
        options_list = roi_select.options.copy()
        options_list.append(text_input_roi_name.value)
        roi_select.options = []
        roi_select.options = options_list
    

def remove_roi_by_name_callback(Event):
    del scan_data[-1][text_input_roi_name.value]
    if text_input_roi_name in roi_select.options:
        options_list = roi_select.options.copy()
        options_list.append(text_input_roi_name.value)
        roi_select.options = []
        roi_select.options = options_list
        roi_select.values = []
  


add_roi_button = pn.widgets.Button(name='Add ROI', button_type='primary')
add_roi_button.on_click(add_roi_callback)

remove_roi_by_name_button = pn.widgets.Button(name='Remove ROI by name', button_type='primary')
remove_roi_by_name_button.on_click(remove_roi_by_name_callback)




## Main callback routine where the data processing takes place ##

In [ ]:
@callback
def main_loop(in_data):
    
    #global tof_accu
    global x_axe
    global scan_data
    global scan_table
    global roi_select
    
    if not in_data['type'] in [ChannelDataType.TOF.value, ChannelDataType.COMMAND.value]:
        return

    if in_data['type'] == ChannelDataType.TOF.value:
        
        last_record_data = scan_data[-1]
        
        tof_data = in_data['data']
        mass_series = calculate_mass_from_tof(np.copy(tof_data['tof']*1e6)) #timeseries is in microseconds, mass calibration
        
        
        last_record_data['full_range']['roi_H_VMI'] += np.histogram2d(tof_data['x'], tof_data['y'], bins=bins_VMI)[0]
        for roi_key, roi_val in last_record_data.items():
            roi_indxs = np.logical_and(mass_series>roi_val['tof_roi'][0], mass_series<roi_val['tof_roi'][1])
            roi_val['roi_H_VMI'] += np.histogram2d(tof_data['x'][roi_indxs], tof_data['y'][roi_indxs], bins=bins_VMI)[0]


        if roi_select.value == ['full_range']:
            roi_H_VMI = last_record_data['full_range']['roi_H_VMI']
        else:
            roi_H_VMI = np.zeros((255, 255), dtype=float)
            for roi_key, roi_val in last_record_data.items():
                if roi_key == 'full_range':
                    continue
                if roi_key in roi_select.value:
                    roi_H_VMI += roi_val['roi_H_VMI']
        pipe_image.send(roi_H_VMI)

        last_record_data['full_range']['roi_tof_histo'] += np.histogram(mass_series, bins=bins_series)[0]/len(np.unique(tof_data['nr']))
        series_data = last_record_data['full_range']['roi_tof_histo']
        pipe_tof.send(series_data/max(series_data.max(),1))
        
        last_record_data['full_range']['n_tiggers'] += len(np.unique(tof_data['nr']))
        return


    if in_data['type'] == ChannelDataType.COMMAND.value:  
        
        if in_data['data'] == Commands.START_RECORD.value:
            # init new record data with all ROIs
            newRecord = copy.deepcopy(scan_data[-1])
            for roi_name, roi_record in newRecord.items():
                roi_record['roi_tof_histo'] = np.zeros((len(bins_series)-1), dtype=float)
                roi_record['roi_H_VMI'] =  np.zeros((255, 255), dtype=float)
                roi_record['n_tiggers'] = 0
            scan_data.append(newRecord)

        elif in_data['data'] == Commands.STOP_RECORD.value:
            # fill data frame with data from ROIs
            
            vals = []
            roi_names = []
            for roi_name, roi_record in scan_data[-1].items():
                roi_names.append(roi_name)
                if roi_name == 'full_range':
                    continue
                val = 0
                if roi_record['n_tiggers'] > 0:
                    pix_roi = roi_record['spatial_roi']
                    val = roi_record['roi_H_VMI'][pix_roi[0][0]:pix_roi[1][0], pix_roi[0][1]:pix_roi[1][1]]
                    val = np.sum(val)/roi_record['n_tiggers']
                    vals.append(val)

            if scan_table.empty:
                scan_table = pd.DataFrame(columns=roi_names)
            scan_table.loc[len(scan_table.index)] = [len(scan_data)] + vals
                    
            #pipe_roi_scan.send(scan_table)
        return

## Panel GUI, server start ##

In [ ]:
selected_rois = None

def make_roi_scan_plot(data):
    #got dataframe
    if(data.empty):
        return hv.Curve(data=[0])
    
        
    roi_indxs = [roi_select.options.index(i) for i in roi_select.value]

    xmax = np.max(data[:,0])
    xmin = np.min(data[:,0])
    ymax = np.max(data[:,np.array(roi_indxs)+1])
    ymin = np.min(data[:,np.array(roi_indxs)+1])

    if xmax==xmin:
        xlimits = (xmax-1,xmax+1,)
    else:
        xrange = xmax-xmin
        xlimits = (xmin - xrange*0.1,xmax+xrange*0.1,)

    if ymax==ymin:
        ylimits = (ymax-1,ymax+1,)
    else:
        yrange = ymax-ymin
        ylimits = (ymin - yrange*0.1,ymax+yrange*0.1,)

    colors=[i[2] for i in list(Element_ROI.values())]
    curves=[hv.Curve((data[:,0],data[:,i+1])).opts(xlabel='scan num.', ylabel='amplitude, [au]', width=600, height=450, show_grid=True, tools=['hover'], 
                                                   color=colors[i], xlim=xlimits, ylim=ylimits, axiswise=True, framewise=True,)*hv.Scatter((data[:,0],data[:,i+1])).opts(xlabel='scan num.', 
                                                   ylabel='amplitude, [au]', width=600, height=450, show_grid=True, tools=['hover'], color=colors[i], xlim=xlimits, ylim=ylimits, axiswise=True, 
                                                   framewise=True, size=10) for i in roi_indxs]

    
    
def make_image_plot(data):
    #print('got image data')
    #print(data)
    p = hv.Image((range(256), range(256), data.T), bounds=[bins_VMI[0][0], bins_VMI[1][0], bins_VMI[0][-1], \
                                        bins_VMI[1][-1]],).opts(xlabel='x', ylabel='y', axiswise=True, \
                                        logz=True, clim=(1e-3, None), cmap="jet", frame_height=500, \
                                        frame_width=500)
    return p

def make_tof_histo_plot(data):
    global roi_select, selected_rois
    
    last_roi_data = scan_data[-1]
 
    selected_rois = [(last_roi_data[i]['tof_roi'], last_roi_data[i]['roi_color']) for i in roi_select.value if i != 'full_range']
    
    if mass_calibration_data == None:
        xlabel='ToF (µs)'
    else:
        xlabel='Mass (amu)'
        
    ylabel='Normalized amplitude'
    
    p_log = hv.Curve((x_axe, data)).opts(xlabel=xlabel, ylabel=ylabel, axiswise=True, height=400,\
                                     width=1200, show_grid=True, tools=['hover'], ylim=(1e-7,1),xlim=(x_axe[0],\
                                     x_axe[-1]), logy=TOF_YLOG)
    
    
    #rng = hv.streams.RangeY(source=p)
    
    mass_rois = [hv.Rectangles((i[0], 1e-7, i[1], 1)).opts(alpha=0.3, color=c) for i, c in selected_rois]
    #mass_rois = [hv.Rectangles((i[0], rng[0], i[1], rng[1])).opts(alpha=0.3, color=c) for i, c in selected_rois]
    return p_log * hv.Overlay(mass_rois)

image_tof_panel = pn.Row(pn.Column(roi_select, reset_button),
                         pn.Column(hv.DynamicMap(make_image_plot, streams=[pipe_image]),
                                   hv.DynamicMap(make_tof_histo_plot,  streams=[pipe_tof])))

scan_panel = pn.Row(hv.DynamicMap(make_roi_scan_plot, streams=[pipe_roi_scan]))

mass_calibrate_panel = pn.Row(pn.Column(time_point1, time_point2, mass_point1, mass_point2,\
                                mass_calibrate_button, reset_calibration_button, mass_calib_string),
                                pn.Column(hv.DynamicMap(make_tof_histo_plot,\
                                                                            streams=[pipe_tof])))

roi_panel = pn.Row(pn.Column(m1_point, m2_point, x1_point, x2_point, y1_point,\
                            y2_point,text_input_roi_name, colorpicker, add_roi_button,\
                            remove_roi_by_name_button), pn.Column(hv.DynamicMap(make_image_plot,\
                            streams=[pipe_image]), hv.DynamicMap(make_tof_histo_plot,  streams=[pipe_tof])))


   
pn.serve( pn.Tabs(( 'Image&Spectrum' , image_tof_panel),
                  ('Scan', scan_panel),
                  ('ROI', roi_panel),
                  ('Mass calibr.', mass_calibrate_panel),
                  ('New panel', hv.Curve([])),
                  dynamic=True))

#         port=SERVE_PORT)

roi_select = pn.widgets.MultiChoice(
    name='ToF/mass ROI', value=[],
    options=['1', '2'],)
roi_select 

roi_select.value = []
roi_select.options = ['7']

In [ ]:
#roi_select.param.update()

In [ ]:
#roi_select.params

In [ ]:
#scan_data[-1].keys()

In [ ]:
#roi_select.values=['Oxygen']

In [ ]:
#roi_select.options = ['full_range', 'Oxygen']

In [ ]:
x=(0,150)
y=(90,256)


In [ ]:
hv.Image((range(256), range(256), scan_data[-1]['full_range']['roi_H_VMI'].T)).opts(xlabel='x', ylabel='y', axiswise=True, logz=True, clim=(1e-3, None), cmap="jet", frame_height=500, frame_width=500)

In [ ]:
x = (110,160)
y = (0,256)

hv.Image((range(x[0],x[1]), range(y[0],y[1]), scan_data[-1]['full_range']['roi_H_VMI'][x[0]:x[1], y[0]:y[1]].T )).opts(xlabel='x', ylabel='y', axiswise=True, logz=True, clim=(1e-3, None), cmap="jet", frame_height=500, frame_width=500)

In [ ]:
scan_table

In [ ]:
scan_data[-1]